First import pyscf.
You can install pyscf with the command:

In [ ]:
try:
    from pyscf import gto
    print("Found pyscf")
except:
    print("Can't import, installing via pip")
    !pip3 install pyscf
    from pyscf import gto

Now we build our basis set.

The basis set is in the form:

 $\psi _{STO-nG} = c_{1}\phi_{1} + \cdots c_{n}\phi_{n}$
 
where  $\phi_{n} = \frac{2\alpha_{n}}{\pi} ^{\frac{3}{4}} e^{-\alpha_{n}r^{2}}$

In [ ]:
cpar = '1'
apar = '0.01' #alpha parameter
basis = '''
H    S
      ''' + apar + ' ' + cpar
singlegaussianbasis = {'H': gto.basis.parse(basis)}

Now we build our hydrogen atom which we give the point coordinates at the origin

In [ ]:
mol = gto.Mole()
mol.build(atom = 'H 0.0 0.0 0.0',
          basis = singlegaussianbasis, #sto-2g aug-cc-pVQZ
          spin = 1, #written in 2S not 2S+1
          unit = 'angstrom')

Now we run the calculation and build our object

In [ ]:
mf = mol.RHF().run()

Finally we print the energy in atomic units (hartrees) and in electron volts

In [ ]:
print(f'{mf.e_tot} Hartrees')
print(f'{mf.e_tot * 27.2114079527} eV')

1. Build a loop that considers our single gaussian representation of the basis set with different values for $\alpha$, from 0.01 to 1 with 0.01 spacing, to describe the total energy of the system. Plot the total energy of the system as a function of $\alpha$.
2. Report the optimal value of $\alpha$ for a single gaussian function basis set to describe a hydrogen atom.
3. Explain why changing the value of the $c$ parameter does not change the basis set in this scenario.
4. Consider the optimised STO-nG basis set family for n = 2-6. Plot the energy of the hydrogen atom system with increasing number of basis functions.
5. The aug-cc-pVQZ is one of the most complete basis sets, considering 46 functions for the hydrogen atom. Report the total energy/HF energy of this system. Comment on how this value compares to the true total electronic energy of a hydrogen atom (0.5 hartrees)

## Solution

### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def build_gaussian_basis(cpar=1, apar=0.01):
    '''Build a single gaussian basis with variable c-parameter and alpha parameter.
    The c-parameter does not change the behaviour of the function, but can be set here anyways.
    '''
    basis = '''
    H    S
        ''' + str(apar) + ' ' + str(cpar)
    singlegaussianbasis = {'H': gto.basis.parse(basis)}
    return singlegaussianbasis

def build_molecule(basis):
    '''Build a pyscf molecule with a single H atom.
    '''
    mol = gto.Mole()
    mol.build(
        atom = 'H 0.0 0.0 0.0',
        basis = basis, #sto-2g aug-cc-pVQZ
        spin = 1, #written in 2S not 2S+1
        unit = 'angstrom'
    )
    return mol

### Single Gaussian
1. Run for set of alphas using np.arange 
2. Optimal alpha value by finding minimum energy over all alphas
3. Answer for c-parameter

In [ ]:
# Factor for converting Hartree to eV
h_to_ev = 27.2114079527

# Run for different alphas and fill energy array (could be done with lists too).
alphas = np.arange(1e-2, 1e0, 0.01)
energies = np.zeros(alphas.shape, dtype=np.float32)
for ii_alpha, alpha in enumerate(alphas):
    mol = build_molecule(basis=build_gaussian_basis(apar=alpha))
    mf = mol.RHF().run()
    energies[ii_alpha] = mf.e_tot * 27.2114079527

# optimum alpha is at minimum energy
alpha_opt = alphas[np.argmin(energies)]

In [ ]:
print("c parameter in this case won't change the behavior since for a single function in the basis it will be normalised away anyway.")
fig, ax = plt.subplots(1, 1, figsize=(6, 4))

ax.plot(alphas, energies, label='energy gaussian')
ax.hlines(-0.5*h_to_ev, xmin=0, xmax=1, color='k', label='-0.5 Hartree')
ax.vlines(alpha_opt, ymin=-14, ymax=-3, color='r', label=r'$\alpha_{opt}=%.2f$'%alpha_opt)

ax.set_title('H atom energy for different alpha with single gaussian basis')
ax.set_xlabel(r'$\alpha$')
ax.set_ylabel('total energy [eV]')
ax.set_xscale('log')
ax.legend()

plt.show()

### Other Basis Sets
4. Have to hand import STO-nG sets from [basis set exchange](https://www.basissetexchange.org/) since they are not in pyscf.
5. This is actually part of pyscf.

In [ ]:
# The content of this dict is just copy and pasted from the stack exchange
# Could be a static file too, but it's not so large so why not here
sto_sets = {
    "sto_2g": """
BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (2s) -> [1s]
H    S
      0.1309756377E+01       0.4301284983E+00
      0.2331359749E+00       0.6789135305E+00
END
""",
    "sto_3g":"""
BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (3s) -> [1s]
H    S
      0.3425250914E+01       0.1543289673E+00
      0.6239137298E+00       0.5353281423E+00
      0.1688554040E+00       0.4446345422E+00
END
""",
    "sto_4g":"""
BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (4s) -> [1s]
H    S
      0.8021420155E+01       0.5675242080E-01
      0.1467821061E+01       0.2601413550E+00
      0.4077767635E+00       0.5328461143E+00
      0.1353374420E+00       0.2916254405E+00
END
""",
    "sto_5g":"""
BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (5s) -> [1s]
H    S
      0.1738354739E+02       0.2214055312E-01
      0.3185489246E+01       0.1135411520E+00
      0.8897299079E+00       0.3318161484E+00
      0.3037874103E+00       0.4825700713E+00
      0.1144784984E+00       0.1935721966E+00
END
""",
    "sto_6g":"""
BASIS "ao basis" SPHERICAL PRINT
#BASIS SET: (6s) -> [1s]
H    S
      0.3552322122E+02       0.9163596281E-02
      0.6513143725E+01       0.4936149294E-01
      0.1822142904E+01       0.1685383049E+00
      0.6259552659E+00       0.3705627997E+00
      0.2430767471E+00       0.4164915298E+00
      0.1001124280E+00       0.1303340841E+00
END
"""
}

In [ ]:
# Build the moleculte with a different basis everytime and run
ns_sto = np.arange(2, 7, dtype=np.int8)
sto_energies = np.zeros(ns_sto.shape, dtype=np.float32)
for ii_n, n_sto in enumerate(ns_sto):
    mol = build_molecule(basis={'H': gto.basis.parse(sto_sets['sto_%ug'%n_sto])})
    mf = mol.RHF().run()
    sto_energies[ii_n] = mf.e_tot * 27.2114079527
    print("STO-%uG energy: %f eV"%(n_sto, sto_energies[ii_n]))


In [ ]:
# Finally also run for aug-cc-pVQZ basis
mol = build_molecule(basis={'H': 'aug-cc-pVQZ'})
mf = mol.RHF().run()
sto_energies[ii_n] = mf.e_tot * 27.2114079527
# Pretty much exact